In [1]:
import datetime
import json
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
import sys
import tensorflow as tf

from collections import Counter
from imblearn.over_sampling import SMOTE
from keras import Input
from keras import layers
from keras import models
from keras import regularizers
from keras import utils
from keras.utils import to_categorical
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate # to run validation on multiple metrics
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras.models import save_model


%matplotlib inline

# Preparation Steps

In [2]:
# folders
home = os.getenv("HOME")
nlp_repo = os.path.join(home, 'git/nlp-product-sentiment-classification')

# data
train_csv_path = os.path.join(nlp_repo, 'data/03_processed/Train.csv')
train_descr = pd.read_csv(train_csv_path)

test_csv_path = os.path.join(nlp_repo, 'data/03_processed/Test.csv')
test_descr = pd.read_csv(test_csv_path)

# encoded tokens
preprocessed_corpus_path_TF = os.path.join(
    nlp_repo, 'data/03_processed/product_descr_preprocessed_TF.p')

preprocessed_corpus_path_TF_oh = os.path.join(
    nlp_repo, 'data/03_processed/product_descr_preprocessed_TF_oh.p')

# Parameters

In [ ]:
logging = True

saving = True

PARAMS = {

    # Define experiment name:
    'experiment_name': 'all_models_no_counter_overfitting',

    # List of models
    'model_type': ['bow', 'fc_emb', 'fc_transf', 'lstm', 'lstm_transf', 'conv1d', 'conv1d_transf'],

    # Parameters general:
    'number_of_classes': len(np.unique(train_descr['Sentiment'])),
    'n_splits': 4,
    'seed': 42,
    'batch_size': 32,  # [16, 32, 64],
    'epochs': 15,
    'hidden_units': 32,  # [16, 32, 64],

    # NLP Parameters
    # max_words = vocabulary size = our samples - number of most frequent words.
    # I set it to 10.000, although in this case there are less.
    # I do this to parametise the code.
    # Aleternatively, I can set it to the length of our vocabulary = word_index
    'max_words': 10000,

    # embedding_size = embedding dimensionality
    'embedding_size': 30,  # [10, 20, 30, 100],

    # Cross-fold validation:
    'k': 4,
}

logdir = f'logs/experiments/{PARAMS["experiment_name"]}_' + \
    datetime.datetime.now().strftime("%Y_%m_%d-%H:%M")
logdir_tb = f'logs/tensorboard/experiments/{PARAMS["experiment_name"]}_' + \
    datetime.datetime.now().strftime("%Y_%m_%d-%H:%M")

# create logging folder and tensorboard callback function
if logging:
    print(f'Log results to {logdir}')
    if not os.path.exists(logdir):
        os.makedirs(logdir)

    tensorboard_callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir_tb)]

else:
    logdir = ''
    logdir_tb = ''
    tensorboard_callbacks = []

# Train-Test Split

In [11]:
def train_test_split(data, labels):
    dimensions_labels = PARAMS['number_of_classes']

    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)

    data = data[indices]
    labels = np.asarray(labels)
    labels = labels[indices]

    training_samples = int(0.8 * len(data))

    x_train = data[:training_samples]
    y_train = labels[:training_samples]

    x_test = data[training_samples:]
    y_test = labels[training_samples:]

    y_train_oh = tf.one_hot(indices=y_train, depth=dimensions_labels)
    y_train_oh = np.asarray(y_train_oh)

    y_test_oh = tf.one_hot(indices=y_test, depth=dimensions_labels)
    y_test_oh = np.asarray(y_test_oh)

    return x_train, y_train_oh, x_test, y_test_oh

In [12]:
x_train, y_train, x_test, y_test = train_test_split(data, labels)

x_train_oh, y_train_oh, x_test_oh, y_test_oh = train_test_split(
    data_oh, labels_oh)

# Oversample the Minority Classes

In [13]:
def oversample_smote(x_train, y_train):
    oversample = SMOTE()
    x_train, y_train = oversample.fit_resample(x_train, y_train)

    return x_train, y_train


def plot_classes(y_train):
    counter = Counter(y_train)
    for k, v in counter.items():
        per = v / len(y_train) * 100
        print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
    # plot the distribution
    plt.bar(counter.keys(), counter.values())
    plt.show()

# Generate the multi-input model

In [ ]:
hidden_units = params_dict['hidden_units']
dimensions_labels = params_dict['number_of_classes']
max_words = params_dict['max_words']
max_len = params_dict['max_len']
embedding_size = params_dict['embedding_size']

### 1. Text part

In [ ]:
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(max_words+1, embedding_size, input_length=max_len)(text_input)
flatten_text = layers.Flatten()(embedded_text)
encoded_text_1 = layers.Dense(hidden_units, activation='relu')(flatten_text)
encoded_text_2 = layers.Dense(hidden_units, activation='relu')(encoded_text_1)